## **Instalação dos Pacotes**

In [95]:
!uv pip install -U -q langchain langchain-core langchain-openai langchain-community pydantic

## **Configuração das Variáveis de Ambiente**

Configuração da chave de API da OpenAI para autenticação

In [96]:
import os
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

## **Imports dos Pacotes Usados no Notebook**

Importação dos componentes principais do LangChain para construção de aplicações com LLMs

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

## **Inicialização do Modelo LLM**

Criação da instância do modelo ChatGPT que será utilizado em todos os exemplos

In [99]:
llm = ChatOpenAI(model="gpt-4.1-nano")

print("Modelo LLM inicializado com sucesso!")

Modelo LLM inicializado com sucesso!


In [100]:
file_url = "https://raw.githubusercontent.com/renansantosmendes/synapseai_data/refs/heads/main/company_context.txt"

In [101]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(file_url, encoding="utf-8")
docs = loader.load()

In [102]:
from pydantic import BaseModel, Field
from typing import List, Dict

class ProductAnalysis(BaseModel):
    """Associação de um produto aos seus problemas."""
    product_name: str = Field(description="Nome do produto")
    problems: List[str] = Field(description="Lista de problemas identificados para este produto")

class ProductsProblems(BaseModel):
    """Estrutura final para a extração de múltiplos produtos."""
    content: List[ProductAnalysis] = Field(description="Lista de produtos e seus problemas")

In [103]:
llm_structured = llm.with_structured_output(ProductsProblems)

In [104]:
problems_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes de uma empresa gere uma lista de
possíveis problemas que cada produto da empresa pode apresentar do ponto de
vista do usuário final, como por exemplo API instável, sistema demora muito
para responder,
</context>

<company_description>
{company_description}
</company_description>

Considere o produto "{product_name}" e sua descrição "{product_description}" para conseguir gerar problemas mais
contextualizados
""")

In [105]:
products_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes da empresa gere a descrição do
produto e seu nome. O nome deve ser o mais curto possível, sem adicionar a descrição.
O campo descrição deve ser feito de forma separada do nome do produto

A descrição deve conter todos os detalhes técnicos e não técnicos possíveis sobre o produto em questão.

Garanta que nenhuma informação seja criada, use somente o contexto da empresa para extrair as informações
</context>

<company_description>
{company_description}
</company_description>
""")

In [106]:
class ProductEntry(BaseModel):
    """Representa um produto individual da companhia e sua definição oficial."""
    name: str = Field(
        description="O nome oficial do produto (ex: 'Cartão Platinum', 'App Invest', 'CDB Fácil')"
    )
    description: str = Field(
        description="A descrição detalhada das funcionalidades e regras do produto encontrada no texto"
    )

class CompanyProductCatalog(BaseModel):
    """Catálogo completo de produtos extraído dos documentos da empresa."""
    products: List[ProductEntry] = Field(description="Lista de todos os produtos identificados no texto")

In [107]:
structured_llm = llm.with_structured_output(CompanyProductCatalog)
product_chain = products_prompt | structured_llm

## Criação da lista de produtos + descrições

In [108]:
catalog = product_chain.invoke(
    {
        "company_description": docs[0].page_content
    }
)

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=CompanyProductCatalog(pro...itoramento de risco.')]), input_type=CompanyProductCatalog])
  return self.__pydantic_serializer__.to_python(


In [109]:
catalog.products

[ProductEntry(name='FinBrain', description='O FinBrain é uma plataforma de IA generativa projetada para atuar como copiloto financeiro corporativo, especializando-se em análise financeira, contábil e estratégica. Utiliza modelos de linguagem treinados e ajustados com técnicas de Retrieval-Augmented Generation (RAG) para integrar-se com fontes internas como ERPs, data warehouses, planilhas e PDFs contábeis, garantindo respostas fundamentadas em dados reais do cliente. Permite consultas em linguagem natural sobre indicadores financeiros como EBITDA, fluxo de caixa, orçamentos e projeções, facilitando a geração de relatórios, simulações de cenários e análises comparativas de forma clara, explicável e rastreável, sempre apoiando a tomada de decisão sem substituir a responsabilidade final do profissional.'),
 ProductEntry(name='RiskGen', description='O RiskGen é uma plataforma de IA generativa voltada para a gestão de risco, compliance regulatório e auditoria corporativa. Ela utiliza modelo

In [82]:
catalog_dict = {item['name']: item['description'] for item in catalog.model_dump()["products"]}

In [85]:
current_product = "FinBrain"
# catalog_dict.get(current_product)

## Criação do público de cada produto

In [ ]:
customer_prompt = ChatPromptTemplate.from_template("""
<context>
Com base na seguinte descrição e detalhes de uma empresa gere uma descrição
de um público alvo para o produto,
</context>

<company_description>
{company_description}
</company_description>

Considere o produto "{product_name}" e sua descrição "{product_description}" para conseguir gerar problemas mais
contextualizados
""")

## Criação dos problemas para cada produto

In [86]:
problems_structured_llm = llm.with_structured_output(ProductsProblems)
problems_chain = problems_prompt | problems_structured_llm

In [87]:
%%time
problems_response = problems_chain.invoke({
    "product_name" : current_product,
    "product_description": catalog_dict.get(current_product),
    "company_description": docs[0].page_content
})

CPU times: user 15.9 ms, sys: 1.01 ms, total: 16.9 ms
Wall time: 10.4 s


/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=ProductsProblems(content=...quedas de serviço.'])]), input_type=ProductsProblems])
  return self.__pydantic_serializer__.to_python(


In [88]:
problems_response.model_dump()["content"]

[{'product_name': 'FinBrain',
  'problems': ['API instável, causando falhas nas integrações com sistemas externos.',
   'Demora nas respostas para consultas complexas devido ao tamanho dos dados financeiros.',
   'Dificuldade em entender as respostas geradas, especialmente para usuários não técnicos.',
   'Limitações na capacidade de processar documentos muito extensos, resultando em erros ou omissões.',
   'Desafios em obter dados atualizados em tempo real, levando a informações desatualizadas nas análises.',
   'Possíveis falhas na explicabilidade das análises, dificultando a auditoria das decisões tomadas com base nos insights.',
   'Risco de segurança caso não sejam implementadas corretamente as medidas de criptografia e controle de acesso.',
   'Complexidade excessiva na configuração inicial e integração com sistemas ERP, demandando muito suporte técnico.',
   'Dependência excessiva do sistema por parte de usuários, levando a uma possível diminuição da agilidade nas decisões finan

In [89]:
product_problems_dict = {item['product_name']: item['problems'] for item in problems_response.model_dump()["content"]}

In [90]:
product_problems_dict

{'FinBrain': ['API instável, causando falhas nas integrações com sistemas externos.',
  'Demora nas respostas para consultas complexas devido ao tamanho dos dados financeiros.',
  'Dificuldade em entender as respostas geradas, especialmente para usuários não técnicos.',
  'Limitações na capacidade de processar documentos muito extensos, resultando em erros ou omissões.',
  'Desafios em obter dados atualizados em tempo real, levando a informações desatualizadas nas análises.',
  'Possíveis falhas na explicabilidade das análises, dificultando a auditoria das decisões tomadas com base nos insights.',
  'Risco de segurança caso não sejam implementadas corretamente as medidas de criptografia e controle de acesso.',
  'Complexidade excessiva na configuração inicial e integração com sistemas ERP, demandando muito suporte técnico.',
  'Dependência excessiva do sistema por parte de usuários, levando a uma possível diminuição da agilidade nas decisões financeiras.',
  'Baixa performance em ambie

In [93]:
import random

def get_random_element(input_list):
    """Retorna um elemento aleatório de uma lista."""
    if not input_list:
        return None
    return random.choice(input_list)

# Exemplo de uso com seus produtos:
meus_produtos = ["App Invest", "Cartão Platinum", "CDB Fácil", "Seguro Vida"]

# Toda vez que chamar, virá um diferente:
produto_sorteado = get_random_element(meus_produtos)
print(f"Produto escolhido: {produto_sorteado}")

Produto escolhido: Seguro Vida


In [69]:
import pandas as pd
import json
from enum import Enum
from typing import List
from datetime import datetime
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. Definição das Classes com Regras de Negócio
class CriticalityLevel(str, Enum):
    LOW = "Baixa"
    MEDIUM = "Média"
    HIGH = "Alta"
    CRITICAL = "Crítica"

class SupportTicket(BaseModel):
    """Estrutura de um ticket de suporte com dados realistas."""
    customer_name: str = Field(description="Nome completo real (ex: João Carlos de Alencar, Maria Aparecida Souza)")
    customer_email: str = Field(description="E-mail variado (NÃO use @example.com. Use @gmail, @uol, @empresa.com.br, etc)")
    product_name: str = Field(description="Nome do produto do mercado financeiro ou tecnologia")
    issue_description: str = Field(description="Relato do problema em linguagem natural humana")
    opening_date: str = Field(description="Data de abertura (formato ISO YYYY-MM-DD)")
    criticality: CriticalityLevel = Field(description="Nível de urgência")

class TicketBatch(BaseModel):
    """Lote de tickets para exportação."""
    tickets: List[SupportTicket]

# 2. Configuração do Modelo e Prompt Estratégico
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9) # Temperatura alta para mais criatividade nos nomes
structured_llm = llm.with_structured_output(TicketBatch)

prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um gerador de dados sintéticos de alta fidelidade para o mercado brasileiro.

    DIRETRIZES PARA REALISMO:
    - NOMES: Use nomes brasileiros reais e diversos (sobrenomes como Silva, Oliveira, Cavalcanti, Schmidt, Nogueira).
    - E-MAILS: Use domínios como @hotmail.com, @uol.com.br, @terra.com.br, @gmail.com ou domínios de empresas fictícias (@startup.io, @banco.com.br).
    - PROIBIÇÃO: Está proibido o uso do domínio '@example.com'.
    - DATAS: Gere datas aleatórias no ano de 2025.
    - PERSONAS: Varie entre clientes corporativos sérios e usuários comuns informais."""),
    ("human", "Gere {number_of_tickets} tickets de suporte para o produto '{product_name}' com a seguinte descrição '{product_description}' sobre o problema: '{product_problem}'.")
])

# 3. Execução e Conversão
chain = prompt | structured_llm

# Exemplo de uso para seu projeto de NER
input_data = {
    "number_of_tickets": 5,
    "product_name": "FinBrain",
    "product_description":
    "product_problem": "Erro ao tentar liquidar ordem de compra de ações"
}

resultado = chain.invoke(input_data)

# 4. Transformação em Lista de Dicionários e DataFrame
# Usando model_dump para cada objeto da lista
tickets_dicts = [t.model_dump() for t in resultado.tickets]

df = pd.DataFrame(tickets_dicts)

# Exibindo o resultado para conferência
print(df[['customer_name', 'customer_email', 'criticality']])

# Opcional: Salvar para CSV para usar nos seus slides ou treino de modelo
# df.to_csv("tickets_sinteticos.csv", index=False)

                 customer_name                customer_email  \
0        Mariana Silva Pereira       mariana.silva@gmail.com   
1      Roberto Oliveira Santos   roberto.oliveira@uol.com.br   
2            Ana Clara Schmidt         anaclara@terra.com.br   
3   Fernando Nogueira da Silva  fernando.nogueira@startup.io   
4  Sofia Cavalcanti de Freitas    sofia.cavalcanti@gmail.com   

                 criticality  
0      CriticalityLevel.HIGH  
1  CriticalityLevel.CRITICAL  
2    CriticalityLevel.MEDIUM  
3      CriticalityLevel.HIGH  
4      CriticalityLevel.HIGH  


/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=TicketBatch(tickets=[Supp...tyLevel.HIGH: 'Alta'>)]), input_type=TicketBatch])
  return self.__pydantic_serializer__.to_python(


In [70]:
resultado.tickets[3].model_dump()

{'customer_name': 'Fernando Nogueira da Silva',
 'customer_email': 'fernando.nogueira@startup.io',
 'product_name': 'Plataforma de Investimentos Diretos',
 'issue_description': 'Ao tentar liquidar uma ordem de compra, o sistema apresenta um erro. Não consigo finalizar a operação e estou preocupado.',
 'opening_date': '2025-02-18',
 'criticality': <CriticalityLevel.HIGH: 'Alta'>}